In [1]:
from __future__ import division, print_function
import os
import numpy as np
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import pickle
import operator

In [2]:
app = Flask(__name__)

dataset = []
def loadDataset(filename):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break

loadDataset("my.dat")

In [3]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    
    #Method to calculate distance between two instances.
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    print("distane is",distance)
    return distance


In [4]:
def getNeighbors(trainingSet , instance , k):
    distances =[]
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    print("neighbors is ",neighbors)
    return neighbors   

In [5]:
def nearestClass(neighbors):
    classVote ={}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1 
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    print("sorter is ",sorter)
    return sorter[0][0]

In [6]:
print('Model loaded. Check http://127.0.0.1:5000/')

Model loaded. Check http://127.0.0.1:5000/


In [7]:

@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('music.html')

In [8]:
@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['image']
        
        # Save the file to ./uploads
        basepath = "C:/Externship/Flask"
        file_path = os.path.join(basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)
        print(file_path)
        i=1
        
        results = {1: 'blues', 2: 'classical', 3: 'country', 4: 'disco', 5: 'hiphop', 
                   6: 'jazz', 7: 'metal', 8: 'pop', 9: 'reggae', 10: 'rock'}
    
        (rate,sig)=wav.read(file_path)
        print(rate,sig)
        mfcc_feat=mfcc(sig,rate,winlen=0.020,appendEnergy=False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature=(mean_matrix,covariance,0)
        pred=nearestClass(getNeighbors(dataset ,feature , 8))
        
        print("predicted genre = ",pred,"class = ",results[pred])
        return "This song is classified as a "+str(results[pred])

In [9]:

if __name__ == '__main__':
    app.run(threaded = False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [31/Jul/2023 19:49:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Jul/2023 19:49:06] "GET /favicon.ico HTTP/1.1" 404 -


C:/Externship/Flask\uploads\blues.00000.wav
22050 [  240   544   250 ... -1822 -2001 -2103]
distane is 5.0
distane is 5.0
distane is 21.876863227897445
distane is 12.134149561808172
distane is 13.164516962047063
distane is 11.683336955637504
distane is 24.544006272427268
distane is 14.777381462033944
distane is 26.147568227579562
distane is 20.53464334466058
distane is 29.556875722046748
distane is 18.097265248249492
distane is 36.47278047429462
distane is 17.07683897988242
distane is 32.741853147254304
distane is 15.716190369700985
distane is 13.125225087168484
distane is 13.319160653920363
distane is 26.0277783017739
distane is 26.870838946651453
distane is 25.34772292398776
distane is 18.85063516345037
distane is 31.874348645839277
distane is 21.391115027784984
distane is 33.2664228675076
distane is 30.18038332291279
distane is 26.477614748181907
distane is 48.634869686712925
distane is 40.783542735528854
distane is 53.45885929489225
distane is 27.51548948927438
distane is 35.073553

127.0.0.1 - - [31/Jul/2023 19:49:20] "POST /predict HTTP/1.1" 200 -


distane is 14.682763577955217
distane is 11.949228083998648
distane is 12.764652379105293
distane is 15.294426219294262
distane is 17.539021258965846
distane is 21.604832292464913
distane is 17.996407407470684
distane is 15.969253486255788
distane is 19.51034849244545
distane is 21.73897529693101
distane is 10.502757290177001
distane is 16.771664379950654
distane is 14.709535387395682
distane is 15.171074956996264
distane is 16.14059031304312
distane is 12.981741041311668
distane is 20.971010047602363
distane is 11.405431234499254
distane is 18.12417386555293
distane is 30.147349742082653
distane is 19.769260101041553
distane is 15.909094956710192
distane is 11.81747063105535
distane is 12.071767263247636
distane is 23.215741068659057
distane is 23.326248041672386
distane is 18.94159603009431
distane is 26.96228343422206
distane is 17.161208780859074
distane is 30.2903350737733
distane is 12.102700413403618
distane is 18.25155624630394
distane is 21.07331047405465
distane is 46.8849175

127.0.0.1 - - [31/Jul/2023 19:49:42] "POST /predict HTTP/1.1" 200 -


C:/Externship/Flask\uploads\hiphop.00005.wav
22050 [-13387 -11509 -10099 ...  -7985  -4208  -3139]
distane is 28.79607151988563
distane is 20.223007455595017
distane is 57.62431845489145
distane is 23.067163755767517
distane is 34.06128201507389
distane is 21.41773460686983
distane is 65.07795623235921
distane is 26.305025706802695
distane is 60.77376442013265
distane is 39.87778160250448
distane is 62.34746249979017
distane is 30.132051869769725
distane is 100.84137238731111
distane is 35.1502304279027
distane is 69.85264251339783
distane is 32.523087891955576
distane is 28.206887265052053
distane is 16.83865555349814
distane is 71.10555006135627
distane is 63.070415594873026
distane is 71.84594639413821
distane is 36.31563664103193
distane is 94.53601932462561
distane is 29.641770369650978
distane is 80.95483188417029
distane is 76.93852856137977
distane is 69.67416890092208
distane is 202.18552165188606
distane is 112.29805735563772
distane is 138.61532027068685
distane is 81.343656

127.0.0.1 - - [31/Jul/2023 19:49:57] "POST /predict HTTP/1.1" 200 -


distane is 42.85583007524663
distane is 30.935316186110185
distane is 53.29334944556378
distane is 50.20802772355739
distane is 49.98159020160846
distane is 63.89993760821929
distane is 67.82039437060207
distane is 94.1628235467516
distane is 70.66257320005529
distane is 55.85882236432457
distane is 59.169895713172764
distane is 91.8408223011725
distane is 47.992653071891006
distane is 90.38229554462637
distane is 72.90607681205394
distane is 139.21877399727464
distane is 53.211548844961754
distane is 25.473578767819525
distane is 31.82150915693778
distane is 23.586354622447274
distane is 18.90625269683342
distane is 74.12226666717527
distane is 22.83941684160895
distane is 27.289573206640796
distane is 16.665751142663375
distane is 23.278308693405975
distane is 23.2812998488441
distane is 54.67479748670138
distane is 32.975328107274386
distane is 46.829739036150464
distane is 27.18691170155276
distane is 16.291287451080255
distane is 14.13511946741561
distane is 20.996305626229763
dis